In [1]:
%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES=6

env: CUDA_VISIBLE_DEVICES=6


In [2]:
from tqdm import tqdm
from copy import copy
import itertools
import os, sys
import yaml
import json
import gc
import argparse
import functools
import numpy as np
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed.fsdp import FullyShardedDataParallel as FSDP
from torch.utils.data.distributed import DistributedSampler
from torch.distributed.fsdp.wrap import transformer_auto_wrap_policy
from transformers.models.llama.modeling_llama import LlamaDecoderLayer
from datasets import Dataset

sys.path.append(os.path.abspath('/homes/80/anya/Documents/llm_tiny_ideas/coconut-outer/coconut'))
from utils import Config, set_seed
from coconut import Coconut
from dataset import get_dataset, get_question_latent_dataset, get_cot_latent_dataset, MyCollator


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
# rank = int(os.environ["RANK"])
# print(rank)
# world_size = int(os.environ["WORLD_SIZE"])
# print(world_size)

/homes/80/anya/anaconda3/envs/coconut-1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


# Config

In [5]:
config_file = "../args/gsm_coconut.yaml"
load_dir = "../checkpoints/baseline/gsm-coconut/checkpoint_25"

### Load config
with open(config_file) as f:
        config_dict = yaml.safe_load(f)
print("Config:", config_dict)
configs = Config(config_dict)

Config: {'project': 'coconut', 'save_path': './checkpoints/baseline', 'name': 'gsm-coconut', 'only_eval': False, 'coconut': True, 'cot': False, 'no_thoughts': False, 'no_cot': False, 'c_thought': 2, 'epochs_per_stage': 3, 'max_latent_stage': 3, 'pad_latent_to_max': True, 'save_only_improve': False, 'uniform_prob': 0.0, 'model_id': 'openai-community/gpt2', 'load_model_path': './checkpoints/baseline/gsm-cot/checkpoint_16', 'seed': 0, 'resume': 3, 'bf16': False, 'train_path': 'data/gsm_train.json', 'val_path': 'data/gsm_valid.json', 'reset_optimizer': True, 'batch_size_training': 32, 'debug': False, 'gradient_accumulation_steps': 1, 'num_epochs': 25, 'lr': 0.0001, 'weight_decay': 0.01}


In [ ]:
configs

# Model

In [127]:


### Load weights
saved_weights = torch.load(load_dir, map_location=torch.device(0))
for k, v in saved_weights.items():
    print(k, v.shape)

### Load base model architecture and tokenizer
model = AutoModelForCausalLM.from_pretrained(configs.model_id)
# print(f"{model=}")
tokenizer = AutoTokenizer.from_pretrained(configs.model_id)
# print(f"{tokenizer=}")
### Add new tokens to the tokenizer
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_tokens("<|start-latent|>")
tokenizer.add_tokens("<|end-latent|>")
tokenizer.add_tokens("<|latent|>")
latent_id = tokenizer.convert_tokens_to_ids("<|latent|>")
start_id = tokenizer.convert_tokens_to_ids("<|start-latent|>")
end_id = tokenizer.convert_tokens_to_ids("<|end-latent|>")
print(f"{tokenizer=}")
### Add new tokens to the model
model.resize_token_embeddings(len(tokenizer))
embeddings = model.get_input_embeddings()
target_id = tokenizer.convert_tokens_to_ids("<<")
# initialize the new token embeddings with a known token
# it helps stablize the training
for token_id in [latent_id, start_id, end_id]:
    target_embedding = embeddings.weight.data[token_id]
    embeddings.weight.data[token_id] = target_embedding
    # The input embeddings and lm heads are tied in GPT2. So the code below is not necessary
    lm_head = model.lm_head
    lm_head.weight.data[token_id] = lm_head.weight.data[target_id]
print(f"{model=}")

### Make coconut model and load weights
model = Coconut(model, latent_id, start_id, end_id, tokenizer.eos_token_id)
print(model.load_state_dict(saved_weights, strict=False))
model = model.to(device)

if configs.bf16:
    print("Using bfloat16")
    model.to(torch.bfloat16)

/tmp/ipykernel_4027419/1751632661.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  saved_weights = torch.load(load_dir, map_location=torch.device(0))


base_causallm.transformer.wte.weight torch.Size([50260, 768])
base_causallm.transformer.wpe.weight torch.Size([1024, 768])
base_causallm.transformer.h.0.ln_1.weight torch.Size([768])
base_causallm.transformer.h.0.ln_1.bias torch.Size([768])
base_causallm.transformer.h.0.attn.c_attn.weight torch.Size([768, 2304])
base_causallm.transformer.h.0.attn.c_attn.bias torch.Size([2304])
base_causallm.transformer.h.0.attn.c_proj.weight torch.Size([768, 768])
base_causallm.transformer.h.0.attn.c_proj.bias torch.Size([768])
base_causallm.transformer.h.0.ln_2.weight torch.Size([768])
base_causallm.transformer.h.0.ln_2.bias torch.Size([768])
base_causallm.transformer.h.0.mlp.c_fc.weight torch.Size([768, 3072])
base_causallm.transformer.h.0.mlp.c_fc.bias torch.Size([3072])
base_causallm.transformer.h.0.mlp.c_proj.weight torch.Size([3072, 768])
base_causallm.transformer.h.0.mlp.c_proj.bias torch.Size([768])
base_causallm.transformer.h.1.ln_1.weight torch.Size([768])
base_causallm.transformer.h.1.ln_1.b

In [141]:
dataset_dir = "../data/my_data/gsm8k"
dataset = Dataset.load_from_disk(f"{dataset_dir}/test.bin")
print(f"{dataset=}")
print(f"{dataset[0]=}")

dataset=Dataset({
    features: ['prompt', 'reasoning', 'search_string', 'answer'],
    num_rows: 1319
})
dataset[0]={'prompt': 'Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers\' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers\' market? Let\'s think step by step and output the final answer after "####".', 'reasoning': 'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n', 'search_string': '####', 'answer': '18'}


In [129]:
### Load base model architecture and tokenizer
model_id2 = "Qwen/Qwen2.5-0.5b"
model2 = AutoModelForCausalLM.from_pretrained(model_id2)
# print(f"{model=}")
tokenizer2 = AutoTokenizer.from_pretrained(model_id2)
# print(f"{tokenizer=}")
### Add new tokens to the tokenizer
print(f"{tokenizer2.pad_token=}")
tokenizer2.pad_token = tokenizer2.eos_token
# tokenizer2.add_tokens("<|start-latent|>")
# tokenizer2.add_tokens("<|end-latent|>")
# tokenizer2.add_tokens("<|latent|>")
# latent_id = tokenizer.convert_tokens_to_ids("<|latent|>")
# start_id = tokenizer.convert_tokens_to_ids("<|start-latent|>")
# end_id = tokenizer.convert_tokens_to_ids("<|end-latent|>")
# print(f"{tokenizer=}")
# ### Add new tokens to the model
# model.resize_token_embeddings(len(tokenizer))
# embeddings = model.get_input_embeddings()
# target_id = tokenizer.convert_tokens_to_ids("<<")
# # initialize the new token embeddings with a known token
# # it helps stablize the training
# for token_id in [latent_id, start_id, end_id]:
#     target_embedding = embeddings.weight.data[token_id]
#     embeddings.weight.data[token_id] = target_embedding
#     # The input embeddings and lm heads are tied in GPT2. So the code below is not necessary
#     lm_head = model.lm_head
#     lm_head.weight.data[token_id] = lm_head.weight.data[target_id]
# print(f"{model=}")

# ### Make coconut model and load weights
# model = Coconut(model, latent_id, start_id, end_id, tokenizer.eos_token_id)
# print(model.load_state_dict(saved_weights, strict=False))
# model = model.to(device)

# if configs.bf16:
#     print("Using bfloat16")
#     model.to(torch.bfloat16)

tokenizer2.pad_token='<|endoftext|>'


In [149]:
max_length = 100
do_sample = False
top_p = 0.95
top_k = 50
model_id2 = "Qwen/Qwen2.5-0.5b"
# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_id2)
tokenizer = AutoTokenizer.from_pretrained(model_id2)
model = model.to(device)

responses = []
for i in range(3):
    prompts = dataset[0:3]['prompt']
    print(f"{i=}: {prompts=}")
    # Tokenize the input prompt
    input_ids = tokenizer.batch_encode(prompts, return_tensors="pt", return_attention_mask=True).to(device)
    # Generate the model output
    outputs = model.generate(
        input_ids,
        max_length=max_length,
        do_sample=do_sample,
        top_p=top_p,
        top_k=top_k
    )
    # Decode the generated tokens to text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    responses.append(generated_text)


i=0: prompts=['Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers\' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers\' market? Let\'s think step by step and output the final answer after "####".', 'A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take? Let\'s think step by step and output the final answer after "####".', 'Josh decides to try flipping a house.  He buys a house for $80,000 and then puts in $50,000 in repairs.  This increased the value of the house by 150%.  How much profit did he make? Let\'s think step by step and output the final answer after "####".']


AttributeError: 'Qwen2TokenizerFast' object has no attribute 'batch_encode'

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset

def load_and_query_model_from_dataset(
    dataset,
    model_id="Qwen/Qwen2.5-0.5b",
    batch_size=8,
    do_sample=False,
    top_p=0.95,
    top_k=50,
    device=None  # Set device, e.g., "cuda" or "cpu"
):
    """
    Loads a model and tokenizer, extracts prompts from a Hugging Face Dataset,
    tokenizes prompts in batches (with attention masks), sends them to the specified device,
    and generates responses.

    Args:
        dataset (datasets.Dataset): A Hugging Face Dataset object with a "prompt" column.
        model_id (str): The model identifier.
        batch_size (int): Number of prompts to process per batch.
        max_length (int): Maximum length of the generated sequences.
        do_sample (bool): Whether to use sampling (True) or greedy decoding (False).
        top_p (float): Nucleus sampling probability threshold.
        top_k (int): The number of highest probability vocabulary tokens to keep for top-k filtering.
        device (str): Device to run the model on ("cuda" or "cpu"). If None, defaults to cuda if available.

    Returns:
        list of str: A list of generated responses corresponding to each prompt.
    """
    # Determine device if not provided
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    # Load model and tokenizer
    model = AutoModelForCausalLM.from_pretrained(model_id).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    # Extract prompts from the dataset
    prompts = dataset["prompt"]

    responses = []
    # Process prompts in batches
    for i in range(0, 10, batch_size):
        batch_prompts = prompts[i:i + batch_size]
        
        # Tokenize the batch, which automatically generates attention masks
        batch = tokenizer(
            batch_prompts,
            padding=True,
            truncation=True,
            return_tensors="pt",
            padding_side="left",
        )
        
        # Move the inputs to the specified device
        input_ids = batch.input_ids.to(device)
        attention_mask = batch.attention_mask.to(device)
        
        # Generate outputs using the model
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            do_sample=do_sample,
            top_p=top_p,
            top_k=top_k
        )
        
        # Decode the outputs for each prompt in the batch
        batch_responses = [
            tokenizer.decode(output, skip_special_tokens=True) for output in outputs
        ]
        responses.extend(batch_responses)
    
    return responses
    
# Generate responses in batches
responses = load_and_query_model_from_dataset(dataset, batch_size=3)

for prompt, correct_answer, response in zip(dataset["prompt"], dataset["answer"], responses):
    print(f"Prompt: {prompt}\nResponse: {response}\nCorrect answer: {correct_answer}\n\n")


/homes/80/anya/anaconda3/envs/coconut-1/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Prompt: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market? Let's think step by step and output the final answer after "####".
Correct answer: 18
Response: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market? Let's think step by step and output the final answer after "####". To determine how much Janet makes in dollars every day at the farmers' market, we need to follow these steps:

1. Calculate the total number of eggs laid by the ducks in a day.
2. Subtract the number of eggs eaten by Janet from the total number of eggs to find the nu

In [161]:
model_id = "Qwen/Qwen2.5-0.5b"
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)
prompts = dataset["prompt"]

batch_prompts = dataset["prompt"][:3]

# Tokenize the batch, which automatically generates attention masks
batch = tokenizer(
    batch_prompts,
    padding=True,
    truncation=True,
    return_tensors="pt",
    padding_side="left",
).to(device)

outputs = model(**batch)

In [170]:
for k, v in outputs.items():
    print(k)
print(outputs.logits.shape)
for k, v in outputs.past_key_values:
    print(k.shape, v.shape)

logits
past_key_values
torch.Size([3, 80, 151936])
torch.Size([3, 2, 80, 64]) torch.Size([3, 2, 80, 64])
torch.Size([3, 2, 80, 64]) torch.Size([3, 2, 80, 64])
torch.Size([3, 2, 80, 64]) torch.Size([3, 2, 80, 64])
torch.Size([3, 2, 80, 64]) torch.Size([3, 2, 80, 64])
torch.Size([3, 2, 80, 64]) torch.Size([3, 2, 80, 64])
torch.Size([3, 2, 80, 64]) torch.Size([3, 2, 80, 64])
torch.Size([3, 2, 80, 64]) torch.Size([3, 2, 80, 64])
torch.Size([3, 2, 80, 64]) torch.Size([3, 2, 80, 64])
torch.Size([3, 2, 80, 64]) torch.Size([3, 2, 80, 64])
torch.Size([3, 2, 80, 64]) torch.Size([3, 2, 80, 64])
torch.Size([3, 2, 80, 64]) torch.Size([3, 2, 80, 64])
torch.Size([3, 2, 80, 64]) torch.Size([3, 2, 80, 64])
torch.Size([3, 2, 80, 64]) torch.Size([3, 2, 80, 64])
torch.Size([3, 2, 80, 64]) torch.Size([3, 2, 80, 64])
torch.Size([3, 2, 80, 64]) torch.Size([3, 2, 80, 64])
torch.Size([3, 2, 80, 64]) torch.Size([3, 2, 80, 64])
torch.Size([3, 2, 80, 64]) torch.Size([3, 2, 80, 64])
torch.Size([3, 2, 80, 64]) torc

In [ ]:
outputs

In [157]:
for prompt, response, correct_answer in zip(dataset["prompt"], responses, dataset["answer"]):
    print(f"Prompt: {prompt}\nResponse: {response}\nCorrect answer: {correct_answer}\n", "-"*50)

Prompt: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market? Let's think step by step and output the final answer after "####".
Response: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market? Let's think step by step and output the final answer after "####". To determine how much Janet makes in dollars every day at the farmers' market, we need to follow these steps:

1. Calculate the total number of eggs laid by the ducks in a day.
2. Subtract the number of eggs eaten by Janet from the total number of eggs to find the number of eggs sold.


# Dataset

In [130]:
# prepare the ground truth answer and cot for evaluation
val_path = f"../{configs.val_path}"
question_val = [d["question"] for d in json.load(open(val_path))]
answers_val = [d["answer"].replace(",", "").strip() for d in json.load(open(val_path))]
cot_val = ["\n".join(d["steps"]) for d in json.load(open(val_path))]

base_dataset_valid = get_dataset(val_path, tokenizer, max_size=32 if configs.debug else 100000000)

if "gsm" in configs.val_path:
    max_new_tokens = 64
else:
    max_new_tokens = 128

collator = MyCollator(tokenizer, latent_id=latent_id, label_pad_token_id=-100)

Map (num_proc=32): 100%|██████████| 500/500 [00:01<00:00, 447.29 examples/s]


In [131]:
print(f"{base_dataset_valid}")
print(f"{base_dataset_valid[0]}\n")
print(f"Question: {[tokenizer.decode(base_dataset_valid[0]["question_tokenized"])]}")
print(f"Steps: {tokenizer.batch_decode(base_dataset_valid[0]["steps_tokenized"])}")
print(f"Answer: {[tokenizer.decode(base_dataset_valid[0]["answer_tokenized"])]}")

Dataset({
    features: ['idx', 'question_tokenized', 'steps_tokenized', 'answer_tokenized'],
    num_rows: 500
})
{'idx': 0, 'question_tokenized': [7554, 6630, 465, 8701, 284, 362, 8331, 13, 220, 632, 13676, 764, 20, 8331, 583, 1227, 13, 220, 1649, 340, 3011, 284, 604, 8331, 339, 6630, 340, 736, 866, 284, 362, 8331, 13, 220, 632, 1575, 720, 3064, 284, 651, 465, 8701, 2005, 13, 220, 1374, 881, 857, 339, 1414, 583, 614, 30, 198], 'steps_tokenized': [[16791, 19, 12, 17, 28, 17, 4211, 198], [16791, 17, 11757, 20, 28, 19, 4211, 198], [16791, 1065, 14, 19, 28, 18, 4211, 198], [16791, 3064, 9, 18, 28, 6200, 4211, 198]], 'answer_tokenized': [21017, 5867, 50256]}

Question: ['John cuts his grass to 2 inches.  It grows .5 inches per month.  When it gets to 4 inches he cuts it back down to 2 inches.  It cost $100 to get his grass cut.  How much does he pay per year?\n']
Steps: ['<<4-2=2>>\n', '<<2/.5=4>>\n', '<<12/4=3>>\n', '<<100*3=300>>\n']
Answer: ['### 300<|endoftext|>']


In [132]:
epoch = 10
scheduled_stage = (
    0 if (configs.cot or configs.no_cot) else epoch // configs.epochs_per_stage
)
dataset_gen_val = get_question_latent_dataset(
    scheduled_stage,
    base_dataset_valid,
    configs,
    start_id,
    latent_id,
    end_id,
    no_special_marker=configs.cot or configs.no_cot or configs.no_thoughts,
)

dataset_loss_val = get_cot_latent_dataset(
    scheduled_stage,
    base_dataset_valid,
    configs,
    start_id,
    latent_id,
    end_id,
    no_special_marker=configs.cot or configs.no_cot or configs.no_thoughts,
)


Map (num_proc=32): 100%|██████████| 500/500 [00:00<00:00, 967.08 examples/s]


In [133]:
idx_ = 7
print(f"loss: {dataset_loss_val=}")
print(f"eval: {dataset_gen_val=}")

print(f"loss: {dataset_loss_val[idx_]=}")
print(f"eval: {dataset_gen_val[idx_]=}")

print(f"loss: idx: {dataset_loss_val[idx_]['idx']}")
print(f"eval: idx: {dataset_gen_val[idx_]['idx']}")

print(f"loss: input_ids: {[tokenizer.decode(dataset_loss_val[idx_]['input_ids'])]}")
print(f"eval: input_ids: {[tokenizer.decode(dataset_gen_val[idx_]["input_ids"])]}")

attention_mask_loss = dataset_loss_val[idx_]["attention_mask"]
attention_mask_gen = dataset_gen_val[idx_]["attention_mask"]
print(f"loss: attention_mask: len={len(attention_mask_loss)}, sum={sum(attention_mask_loss)}")
print(f"eval: attention_mask: len={len(attention_mask_gen)}, sum={sum(attention_mask_gen)}")

position_ids_loss = dataset_loss_val[idx_]["position_ids"]
position_ids_gen = dataset_gen_val[idx_]["position_ids"]
print(f"loss: position_ids: len={len(position_ids_loss)}, is equal to arange={all([x == y for x, y in zip(position_ids_loss, list(range(len(position_ids_loss))))])}")
print(f"eval: position_ids: len={len(position_ids_gen)}, is equal to arange={all([x == y for x, y in zip(position_ids_gen, list(range(len(position_ids_gen))))])}")

labels_loss = dataset_loss_val[idx_]["labels"]
labels_not_dummy = [x for x in labels_loss if x != -100]
assert [x == y for x, y in zip(labels_loss[-len(labels_not_dummy):], labels_not_dummy)]
print(f"loss: labels: len={len(labels_loss)}, len_not_dummy={len(labels_not_dummy)}, {[tokenizer.decode(labels_not_dummy)]}\n")

loss: dataset_loss_val=Dataset({
    features: ['idx', 'input_ids', 'labels', 'attention_mask', 'position_ids'],
    num_rows: 500
})
eval: dataset_gen_val=Dataset({
    features: ['idx', 'input_ids', 'attention_mask', 'position_ids'],
    num_rows: 500
})
loss: dataset_loss_val[idx_]={'idx': 7, 'input_ids': [22697, 544, 3382, 284, 2822, 617, 8234, 13, 2034, 829, 1575, 720, 17, 11, 35484, 1575, 720, 16, 11, 290, 48389, 1575, 720, 18, 13, 1002, 13067, 544, 24779, 1105, 22514, 11, 604, 35484, 290, 604, 48389, 11, 644, 318, 262, 2811, 1575, 286, 1123, 3704, 286, 8234, 287, 5054, 30, 198, 50257, 50259, 50259, 50259, 50259, 50259, 50259, 50258, 16791, 1731, 10, 19, 10, 1065, 28, 1821, 4211, 198, 16791, 1065, 10, 19, 10, 19, 28, 1238, 4211, 198, 16791, 1821, 14, 1238, 28, 17, 4211, 198, 21017, 362, 50256], 'labels': [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -10

In [134]:
valid_gen_dataloader = torch.utils.data.DataLoader(
    dataset_gen_val,
    num_workers=1,
    pin_memory=True,
    batch_size=1,
    collate_fn=collator,
    # sampler=DistributedSampler(dataset_gen_val, shuffle=False),
)
valid_loss_dataloader = torch.utils.data.DataLoader(
    dataset_loss_val,
    num_workers=1,
    shuffle=False,
    pin_memory=True,
    batch_size=configs.batch_size_training,
    collate_fn=collator,
    # sampler=DistributedSampler(dataset_loss_val, shuffle=False),
)


In [135]:
batch_for_loss = next(iter(valid_loss_dataloader))
batch_for_gen = next(iter(valid_gen_dataloader))
batch_for_loss = {key: batch_for_loss[key].to(device) for key in batch_for_loss.keys() if key != "idx"}
batch_for_gen = {key: batch_for_gen[key].to(device) for key in batch_for_gen.keys() if key != "idx"}

In [136]:
outputs = model(**batch_for_loss)
outputs

input_ids.shape=torch.Size([32, 176])
attention_mask.shape=torch.Size([32, 176])
labels.shape=torch.Size([32, 176])
position_ids.shape=torch.Size([32, 176])
latent_indices=tensor([[  0, 138],
        [  0, 139],
        [  0, 140],
        [  0, 141],
        [  0, 142],
        [  0, 143],
        [  1, 138],
        [  1, 139],
        [  1, 140],
        [  1, 141],
        [  1, 142],
        [  1, 143],
        [  2, 138],
        [  2, 139],
        [  2, 140],
        [  2, 141],
        [  2, 142],
        [  2, 143],
        [  3, 138],
        [  3, 139],
        [  3, 140],
        [  3, 141],
        [  3, 142],
        [  3, 143],
        [  4, 138],
        [  4, 139],
        [  4, 140],
        [  4, 141],
        [  4, 142],
        [  4, 143],
        [  5, 138],
        [  5, 139],
        [  5, 140],
        [  5, 141],
        [  5, 142],
        [  5, 143],
        [  6, 138],
        [  6, 139],
        [  6, 140],
        [  6, 141],
        [  6, 142],
        

Outputs(loss=tensor(10.4648, device='cuda:0', grad_fn=<NllLossBackward0>), inputs_embeds=tensor([[[ 0.0336, -0.0021,  0.0727,  ..., -0.0568,  0.1550,  0.1194],
         [ 0.0336, -0.0021,  0.0727,  ..., -0.0568,  0.1550,  0.1194],
         [ 0.0336, -0.0021,  0.0727,  ..., -0.0568,  0.1550,  0.1194],
         ...,
         [ 0.0336, -0.0021,  0.0727,  ..., -0.0568,  0.1550,  0.1194],
         [ 0.0336, -0.0021,  0.0727,  ..., -0.0568,  0.1550,  0.1194],
         [ 0.0336, -0.0021,  0.0727,  ..., -0.0568,  0.1550,  0.1194]],

        [[ 0.0336, -0.0021,  0.0727,  ..., -0.0568,  0.1550,  0.1194],
         [ 0.0336, -0.0021,  0.0727,  ..., -0.0568,  0.1550,  0.1194],
         [ 0.0336, -0.0021,  0.0727,  ..., -0.0568,  0.1550,  0.1194],
         ...,
         [ 0.0336, -0.0021,  0.0727,  ..., -0.0568,  0.1550,  0.1194],
         [ 0.0336, -0.0021,  0.0727,  ..., -0.0568,  0.1550,  0.1194],
         [ 0.0336, -0.0021,  0.0727,  ..., -0.0568,  0.1550,  0.1194]],

        [[ 0.0336, -0.0021,

In [137]:
# Iterate through the keys and values
for k, v in outputs._asdict().items():
    print(k, v.shape)

loss torch.Size([])
inputs_embeds torch.Size([32, 176, 768])
logits torch.Size([32, 176, 50260])
